# Python 2 HSUTCC: Session 11: Iterator & Generator

## Iterator

### Let's talk about `for`

`for` statement is so magical since it seems to work with multiple data types. How exactly does this magic work? We will cover it in this lecture.

In [ ]:
print('I work with list!')
for element in [1, 2, 3]:
    print(element)

print('\nI work with tuple!')
for element in (1, 2, 3):
    print(element)

print('\nI work with dictionary!')
for key in {'one': 1, 'two': 2}:
    print(key)

print('\nI work with string!')
for char in '123':
    print(char)

To get a glimse of the answer, let's make this silly error.

In [ ]:
for _ in 3:
    pass

`'int' object is not iterable`

Iterable? What a mysterious keywords...

### What is an Iterable?

**Iterable** is an object that defines the method `__next__` which accesses elements in the container one at a time. When there are no more elements, `__next__` raises a `StopIteration` exception.

Aside from iterable, we will also need to talk about iterator. **Iterator** is an object with `__iter__` method which returns an instance of an iterable class. If the class itself implements the `__next__` method, then `__iter__` can just return `self`.

In [ ]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [ ]:
collection = Reverse(data=[1, 2, 3, 4, 5])

for element in collection:
    print(element)

However, `__iter__` can also return an instance of another class whose `__next__` method is implemented.

In [ ]:
import random

class IterCostructor:
    def __iter__(self):
        print('IterCostructor RandomIterator')
        return RandomIterator()

class RandomIterator:
    def __next__(self):
        return random.randint(0, 10)

What will happend here?

In [ ]:
collection = IterCostructor()

for element in collection:
    print(element)

It runs forever......

So, let's get back and talk about `for`.

### How `for` works

Let's use a simple example.

In [ ]:
very_simple_list = [1, 2, 3]

When the `for` keyword is invoked, it calls `iter()` on the container object.

In [ ]:
iterator_list = iter(very_simple_list)

print(iterator_list)

In [ ]:
very_simple_list.__iter__()

Now, `for` can access the elements one by one using the `__next__` method.

In [ ]:
print(next(iterator_list))

And again

In [ ]:
print(next(iterator_list))

And again

In [ ]:
print(next(iterator_list))

And...

In [ ]:
print(next(iterator_list))

Finally, when the collection runs out of elements, a `StopIteration` exception is raise, terminating the `for` loop. All of these steps are packed together as a small `for` loop statement.

In [ ]:
for element in [1, 2, 3]:
    print(element)

For a nerdy team, we can express the `for` loop process as follow

```python
iterable = iter(iterator)
while True:
    try:
        element = next(iterable)
    except StopIteration:
        break
    do_something(element)
```

## Bonus: `in` and `not in` operators

In [ ]:
's' in 'sunday'
's' in ['a', 's', 's']
's' in ('a', 's', 's')
's' in {'a': 1, 's': 2}
's' in 2

Operators `in` and `not in` use a “magic” method `__contains__` which returns `True` if a passed element is contained in a class instance.

By default `__contains__` method is implemented through an iterator protocol:

```python
class object:
    # ...
    def __contains__(self, target):
        for item in self:
            if item == target:
                return True
        return False
```

Hence, the speed of lookup action depends on this `__contains__` method and the collection traversal (`__next__`) process.

In [ ]:
class Kam:
    def __init__(self, data):
        self.data = data
        self.index = -1

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == len(self.data) - 1:
            raise StopIteration
        self.index += 1
        return self.data[self.index]

    def __contains__(self, target):
        if target == 'x':
            return True
        return False

'pizza' in Kam(['yan', 'pizza', 'nugget'])

## Generator

### What does generator generates?

Generators are a simple and powerful tool for creating iterators. They are written like regular functions but use the `yield` statement whenever they want to return data. Each time `next()` is called on it, the generator resumes where it left off (it remembers all the data values and which statement was last executed). An example shows that generators can be trivially easy to create:

In [ ]:
def reverse(data): # [1, 2, 3]
    for index in range(len(data) - 1, -1, -1): # 2, 1, 0
        yield data[index]

In [ ]:
reverse([1, 2, 3])

In [ ]:
x = iter(reverse([1, 2, 3]))
print(next(x))
print(next(x))
next(x)
next(x)

In [ ]:
# index = 2
# wait at yield data[2]
# next()
# return data[2]
# index = 1
# wait at yield data[1]
# next()
# return data[1]
# index = 0
# wait at yield data[0]
# next()
# return data[0]
# no more yeild
# next()
# Error

In [ ]:
for char in reverse([1, 2, 3]):
    print(char)

In [ ]:
for index, element in enumerate(['a','b','c']):
    print(index, element)

Therefore, `enumerate()` is a generator as it is used from Python 1 in the statement:
```python

for index, element in enumerate(lst):
    print(index, element)
```

Now, acts like there is no `enumerate()` generator ever implemented. Please write `custom_enumerate()` generator where each time `next()` is called on the `custom_enumerate()` taking list as an argument, it generates `index` and that index's `element` out.

```python
for item in custom_enumerate(['a', 'b', 'c']):
    print(item)
```
should prints:
```
(0, 'a')
(1, 'b')
(2, 'c')
```

In [ ]:
def custom_generator(data):
    for index in range(len(data)):
        yield index, data[index]

In [ ]:
for element in enumerate({'one': 1, 'two': 2}):
    print(element)

In [ ]:
def custom_generator_v2(data):
    index = 0
    for element in data:
        yield index, element
        index += 1

In [ ]:
class CustomGeneratorV2:
    def __init__(self, data):
        self.data = data
        self.index = -1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == len(self.data) - 1:
            raise StopIteration
        self.index += 1
        return self.index, self.data[self.index]

Anything that can be done with generators can also be done with class-based iterators as described in the previous section. What makes generators so compact is that the `__iter__()` and `__next__()` methods are created automatically.

Another key feature is that the local variables and execution state are automatically saved between calls. This made the function easier to write and much more clear than an approach using instance variables like `self.index` and `self.data`.

In addition to automatic method creation and saving program state, when generators terminate, they automatically raise `StopIteration`. In combination, these features make it easy to create iterators with no more effort than writing a regular function.

In [ ]:
def multi_yield():
    n = 10
    yield_str = "This will print the first string"
    yield yield_str
    yield_str = "This will print the second string"
    yield yield_str

multi_obj = multi_yield()

In [ ]:
multi_obj

In [ ]:
next(multi_obj)

In [ ]:
next(multi_obj)

In [ ]:
next(multi_obj)

Some simple generators can be coded concisely as expressions using a syntax similar to list comprehensions but with parentheses instead of square brackets. These expressions are designed for situations where the generator is used right away by an enclosing function. Generator expressions are more compact but less versatile than full generator definitions and tend to be more memory friendly than equivalent list comprehensions.

In [ ]:
l = [1, 2, 3]

if l:
    print('Yes')

In [ ]:
[i * i for i in range(10)]

In [ ]:
x = (i * i for i in range(10))
print(next(x))
print(next(x))
print(next(x))

In [ ]:
def generate_square(number: int = 10):
    for i in range(number):
        yield i * i

x = generate_square(10)
print(x.__iter__().__next__())
print(x.__next__())
print(next(iter(x)))
print(next(x))

In [ ]:
next(zip([1, 2, 3], ['a', 'b', 'c'], [True, False, True]))

In [ ]:
x_vector = [1, 2, 3]
y_vector = [0.1, 0.5, 0.333]
for result in (x * y for x, y in zip(x_vector, y_vector)):
    print(result)

In [ ]:
list(i * i for i in range(10))

In [ ]:
list(range(10))

In [ ]:
next(iter(range(10)))

# Tasks (Saturday 22 Nov 2025)

1. Pretend there is no way to get i-th element of the list. Also we don't know how to use `for` loop. Given that, print the 3rd element from `['a', 'b', 'c']` list

In [ ]:
sample = ['a', 'b', 'c']

it = iter(sample)
next(it)      # skip 'a'
next(it)      # skip 'b'

third = next(it)   # get 'c'
print(third)

2. Below is a "random" iterator. It'll provide you with random numbers within `for` or `while` loop. You can stop by clicking `stop` button (which would call `KeyboardInterrupt` exception)

In [ ]:
import random

class RandomIterator:
    def __init__(self, range_start, range_end):
        self.range_start = range_start
        self.range_end = range_end

    def __next__(self):
        return random.randint(self.range_start, self.range_end)

    def __iter__(self):
        return self


random_iterator = RandomIterator(1, 10)

for number in random_iterator:
    print(number)

Provide `RandomIterator`'s with `iterations_limit` argument. When loop hits `iterations_limit + 1`-th iteration – raise `StopIteration` exception within `__next__` method

In [ ]:
import random

class RandomIterator:
    def __init__(self, range_start, range_end, iterations_limit):
        self.range_start = range_start
        self.range_end = range_end
        self.iterations_limit = iterations_limit
        self.counter = 0   # track how many times __next__ is called

    def __next__(self):
        if self.counter >= self.iterations_limit:
            raise StopIteration

        self.counter += 1
        return random.randint(self.range_start, self.range_end)

    def __iter__(self):
        return self


3. Right now the instance of a `RandomIterator` becomes useless after single `for` loop. Change `RandomIterator` to be able to call `for` loop any number of times whithout making an instance for each loop.

In [ ]:
import random

class RandomIterator:
    def __init__(self, range_start, range_end, iterations_limit):
        self.range_start = range_start
        self.range_end = range_end
        self.iterations_limit = iterations_limit
        self.counter = 0

    def __iter__(self):
        # reset the iterator every time a new loop starts
        self.counter = 0
        return self

    def __next__(self):
        if self.counter >= self.iterations_limit:
            raise StopIteration

        self.counter += 1
        return random.randint(self.range_start, self.range_end)

4. Write a generator that yield numbers from 1 to 10

In [ ]:
def gen_numbers():
    num = 1
    while num <= 10:
        yield num
        num += 1

for n in gen_numbers():
    print(n)

5. Write a generator that randomly yields "Heads" or "Tails"

In [ ]:
import random

def coin_flip():
    while True:
        yield random.choice(["Heads", "Tails"])

for result in coin_flip():
    print(result)

6. Yield word from the following string: `"Generators use less memory then list comprehensions, since they don't store the results of previous calls"`

In [ ]:
def word_yielder(text):
    words = text.split()
    for w in words:
        yield w

text = "Generators use less memory then list comprehensions, since they don't store the results of previous calls"

for word in word_yielder(text):
    print(word)